In [72]:
from tqdm import tqdm_notebook

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']= 100
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import ujson as json
import os


IDs: MB 307; TK 337 \
https://docs.google.com/spreadsheets/d/15e1K0tg5ponA5R6YQkZfihrShTDLAKf5qeKaoVCiuhQ/edit#gid=0

In [74]:
#Team name, using IDs above: mlcourse_ai_fall2019_team_<id1>_…<id4> 

In [834]:
train_X = pd.read_csv('/Users/m/Insight/MLcourse/Dota/train_features_plus.csv', index_col='match_id_hash')
test_X = pd.read_csv('/Users/m/Insight/MLcourse/Dota/test_features_plus.csv', index_col='match_id_hash')
target =  pd.read_csv('/Users/m/Insight/MLcourse/Dota/targets_plus.csv', index_col='match_id_hash')
y = target['radiant_win']

In [802]:
train_X_ability = pd.read_csv('/Users/m/Insight/MLcourse/Dota/train_features_ability.csv', index_col = 'match_id_hash')
test_X_ability = pd.read_csv('/Users/m/Insight/MLcourse/Dota/test_features_ability.csv', index_col = 'match_id_hash')

In [803]:
#train_X = train_X_old.merge(train_X_ability, how='outer', left_index=True, right_index = True)
#test_X = test_X_old.merge(test_X_ability, how='outer', left_index=True, right_index = True)

In [835]:
full_df = train_X.merge(y, how='outer', left_index=True, right_index=True)
full_df.shape

(39675, 306)

In [602]:
test_X.shape

(10000, 305)

In [603]:
#full_df['r1_gold']

In [836]:
hero_id_list = ['r1_hero_id','r2_hero_id','r3_hero_id','r4_hero_id',
                    'r5_hero_id','d1_hero_id','d2_hero_id','d3_hero_id',
                    'd4_hero_id','d5_hero_id']

In [837]:

hero_ids = full_df[['r1_hero_id','r2_hero_id','r3_hero_id','r4_hero_id',
                    'r5_hero_id','d1_hero_id','d2_hero_id','d3_hero_id',
                    'd4_hero_id','d5_hero_id','radiant_win']]
hero_ids_rad_win = hero_ids[hero_ids['radiant_win'] == True]
hero_ids_rad_lose = hero_ids[hero_ids['radiant_win'] == False]
#hero_ids_rad_win.shape[0] + hero_ids_rad_lose.shape[0]

In [838]:
winning_hero_ids1 = hero_ids_rad_win[['r1_hero_id','r2_hero_id',
                                      'r3_hero_id','r4_hero_id','r5_hero_id']]
winning_hero_ids2 = hero_ids_rad_lose[['d1_hero_id','d2_hero_id',
                                      'd3_hero_id','d4_hero_id','d5_hero_id']]
#winning_hero_ids1.shape[0] + winning_hero_ids2.shape[0]

In [839]:
losing_hero_ids1 = hero_ids_rad_win[['d1_hero_id','d2_hero_id',
                                      'd3_hero_id','d4_hero_id','d5_hero_id']]
losing_hero_ids2 = hero_ids_rad_lose[['r1_hero_id','r2_hero_id',
                                      'r3_hero_id','r4_hero_id','r5_hero_id']]

In [840]:
winning_hero_ids1.rename(columns = {'r1_hero_id':'1_id', 'r2_hero_id':'2_id', 
                              'r3_hero_id':'3_id','r4_hero_id':'4_id',
                                    'r5_hero_id':'5_id'}, inplace = True)
winning_hero_ids2.rename(columns = {'d1_hero_id':'1_id', 'd2_hero_id':'2_id', 
                              'd3_hero_id':'3_id','d4_hero_id':'4_id',
                                    'd5_hero_id':'5_id'}, inplace = True)

In [841]:
losing_hero_ids1.rename(columns = {'d1_hero_id':'1_id', 'd2_hero_id':'2_id', 
                              'd3_hero_id':'3_id','d4_hero_id':'4_id',
                                    'd5_hero_id':'5_id'}, inplace = True)
losing_hero_ids2.rename(columns = {'r1_hero_id':'1_id', 'r2_hero_id':'2_id', 
                              'r3_hero_id':'3_id','r4_hero_id':'4_id',
                                    'r5_hero_id':'5_id'}, inplace = True)

In [842]:
#winning_hero_ids = winning_hero_ids1.merge(winning_hero_ids2, how='outer', 
#                                           left_index=True, right_index=True)
winning_hero_ids = pd.concat([winning_hero_ids1, winning_hero_ids2], axis=0)
winning_hero_ids.shape

(39675, 5)

In [843]:
losing_hero_ids = pd.concat([losing_hero_ids1, losing_hero_ids2], axis=0)
losing_hero_ids.shape

(39675, 5)

In [844]:
winning_hero_counts = winning_hero_ids['1_id'].value_counts().sort_index() + winning_hero_ids['2_id'].value_counts().sort_index() + winning_hero_ids['3_id'].value_counts().sort_index() + winning_hero_ids['4_id'].value_counts().sort_index() + winning_hero_ids['5_id'].value_counts().sort_index()

In [845]:
losing_hero_counts = losing_hero_ids['1_id'].value_counts().sort_index() + losing_hero_ids['2_id'].value_counts().sort_index() + losing_hero_ids['3_id'].value_counts().sort_index() + losing_hero_ids['4_id'].value_counts().sort_index() + losing_hero_ids['5_id'].value_counts().sort_index()

In [846]:
winning_hero_counts = winning_hero_counts.sort_values()
winning_hero_dict = winning_hero_counts.to_dict()

In [847]:
losing_hero_counts = losing_hero_counts.sort_values()
losing_hero_dict = losing_hero_counts.to_dict()

In [848]:
hero_counts_win_minus_lose = winning_hero_counts.sort_index() - losing_hero_counts.sort_index()
diff_hero_dict = hero_counts_win_minus_lose.to_dict()

In [849]:
from collections import Counter
total_games_dict = Counter(winning_hero_dict) + Counter(losing_hero_dict)
hero_id_normalize_dict = {k: (diff_hero_dict[k] / total_games_dict[k]) for k in diff_hero_dict}

In [850]:
#run this cell only once! or else NaNs
for col in hero_id_list:
    full_df[col+'success'] = full_df[col].map(diff_hero_dict)
    train_X[col+'success'] = train_X[col].map(diff_hero_dict)
    test_X[col+'success'] = test_X[col].map(diff_hero_dict)

In [851]:
#run only once!
for col in hero_id_list:
    full_df[col+'norm'] = full_df[col].map(hero_id_normalize_dict)
    train_X[col+'norm'] = train_X[col].map(hero_id_normalize_dict)
    test_X[col+'norm'] = test_X[col].map(hero_id_normalize_dict)

In [852]:
full_df.shape

(39675, 326)

In [853]:
#train_X.columns.values == test_X.columns.values

In [854]:
test_X.shape

(10000, 325)

### Look at hero success, taken k heros per team

Heros are numbered from 1 to 120.

### k=4 at a time

In [626]:
import itertools

In [625]:
#train_X_old[hero_ids].max()
#train_X_old[hero_ids].min()
r_hero_ids = hero_id_list[0:5]
d_hero_ids = hero_id_list[5:10]
hero_ids_num = [i+1 for i in range(120)]

In [627]:
heros_df_r_win = hero_ids[hero_ids['radiant_win']==True]
heros_df_r_lose = hero_ids[hero_ids['radiant_win']==False]

In [636]:
r_heros_df_r_win = heros_df_r_win[r_hero_ids]
r_heros_df_r_lose = heros_df_r_lose[r_hero_ids]
d_heros_df_r_win = heros_df_r_win[d_hero_ids]
d_heros_df_r_lose = heros_df_r_lose[d_hero_ids]

In [637]:
r_heros_df_r_win.values.sort()
r_heros_df_r_lose.values.sort()
d_heros_df_r_win.values.sort()
d_heros_df_r_lose.values.sort()

In [638]:
def make_4tuple(df,x):
    df['{}_hero_4tuple_1'.format(x)] = list(zip(df.iloc[:,0],df.iloc[:,1],df.iloc[:,2],df.iloc[:,3]))
    df['{}_hero_4tuple_2'.format(x)] = list(zip(df.iloc[:,1],df.iloc[:,2],df.iloc[:,3],df.iloc[:,4]))
    df['{}_hero_4tuple_3'.format(x)] = list(zip(df.iloc[:,0],df.iloc[:,1],df.iloc[:,3],df.iloc[:,4]))
    df['{}_hero_4tuple_4'.format(x)] = list(zip(df.iloc[:,0],df.iloc[:,1],df.iloc[:,2],df.iloc[:,4]))
    df['{}_hero_4tuple_5'.format(x)] = list(zip(df.iloc[:,0],df.iloc[:,2],df.iloc[:,3],df.iloc[:,4]))
    df = df[df.columns[-5:]]
    return df

In [639]:
r_heros_df_r_win = make_4tuple(r_heros_df_r_win,'r');
r_heros_df_r_lose = make_4tuple(r_heros_df_r_lose,'r');
d_heros_df_r_win = make_4tuple(d_heros_df_r_win,'d');
d_heros_df_r_lose = make_4tuple(d_heros_df_r_lose,'d');

In [640]:
d_heros_df_r_lose.columns

Index(['d_hero_4tuple_1', 'd_hero_4tuple_2', 'd_hero_4tuple_3',
       'd_hero_4tuple_4', 'd_hero_4tuple_5'],
      dtype='object')

In [641]:
tuple_cols_r = ['r_hero_4tuple_1','r_hero_4tuple_2','r_hero_4tuple_3','r_hero_4tuple_4','r_hero_4tuple_5']
tuple_cols_d = ['d_hero_4tuple_1','d_hero_4tuple_2','d_hero_4tuple_3','d_hero_4tuple_4','d_hero_4tuple_5']

In [642]:
dict_4heros = {}
for i in range(r_heros_df_r_win.shape[0]):
    for j in tuple_cols_r:
        if r_heros_df_r_win[j].iloc[i] not in dict_4heros:
            dict_4heros[r_heros_df_r_win[j].iloc[i]] = 1
        else:
            dict_4heros[r_heros_df_r_win[j].iloc[i]] +=1
for i in range(r_heros_df_r_lose.shape[0]):
    for j in tuple_cols_r:
        if r_heros_df_r_lose[j].iloc[i] not in dict_4heros:
            dict_4heros[r_heros_df_r_lose[j].iloc[i]] = -1
        else:
            dict_4heros[r_heros_df_r_lose[j].iloc[i]] += -1
for i in range(d_heros_df_r_win.shape[0]):
    for j in tuple_cols_d:
        if d_heros_df_r_win[j].iloc[i] not in dict_4heros:
            dict_4heros[d_heros_df_r_win[j].iloc[i]] = 1
        else:
            dict_4heros[d_heros_df_r_win[j].iloc[i]] += -1
for i in range(d_heros_df_r_lose.shape[0]):
    for j in tuple_cols_d:
        if d_heros_df_r_lose[j].iloc[i] not in dict_4heros:
            dict_4heros[d_heros_df_r_lose[j].iloc[i]] = -1
        else:
            dict_4heros[d_heros_df_r_lose[j].iloc[i]] += 1

In [454]:
#max(dict_4heros.values())
#min(dict_4heros.values())
#dict_4heros

Concat 4 data frames, then append new columns to full_df

In [643]:
r_win = r_heros_df_r_win.merge(d_heros_df_r_win,how='outer',left_index=True,right_index=True)
d_win = r_heros_df_r_lose.merge(d_heros_df_r_lose,how='outer',left_index=True,right_index=True)

In [644]:
print(r_win.shape)
print(d_win.shape)

(20826, 10)
(18849, 10)


In [647]:
r_win.head();

In [457]:
#no longer needed
#list_to_drop = ['r{}_hero_id'.format(i) for i in range(1,6)] + ['d{}_hero_id'.format(i) for i in range(1,6)]
#r_win.drop(list_to_drop,axis=1,inplace=True)
#r_win.head(1)

,r_hero_4tuple_1,r_hero_4tuple_2,r_hero_4tuple_3,r_hero_4tuple_4,r_hero_4tuple_5,d_hero_4tuple_1,d_hero_4tuple_2,d_hero_4tuple_3,d_hero_4tuple_4,d_hero_4tuple_5
match_id_hash,,,,,,,,,,
b9c57c450ce74a2af79c9ce96fac144d,"(15, 27, 63, 89)","(27, 63, 89, 96)","(15, 27, 89, 96)","(15, 27, 63, 96)","(15, 63, 89, 96)","(1, 14, 56, 58)","(14, 56, 58, 92)","(1, 14, 58, 92)","(1, 14, 56, 92)","(1, 56, 58, 92)"


In [458]:
#d_win.drop(list_to_drop,axis=1,inplace=True)

In [649]:
tuple_cols = tuple_cols_r + tuple_cols_d
four_tuples = pd.concat([r_win,d_win])
four_tuples.columns

Index(['r_hero_4tuple_1', 'r_hero_4tuple_2', 'r_hero_4tuple_3',
       'r_hero_4tuple_4', 'r_hero_4tuple_5', 'd_hero_4tuple_1',
       'd_hero_4tuple_2', 'd_hero_4tuple_3', 'd_hero_4tuple_4',
       'd_hero_4tuple_5'],
      dtype='object')

In [650]:
full_df.shape

(39675, 326)

In [653]:
full_df_plus4tuple = pd.concat([full_df,four_tuples], axis=1)
train_X_plus4tuple = pd.concat([train_X,four_tuples],axis=1)

In [656]:
full_df_plus4tuple.shape
train_X_plus4tuple.shape

(39675, 335)

In [463]:
#set(full_df_plus4tuple.columns.values).difference(set(full_df.columns.values))

In [657]:
#run only once or else nans
for col in tuple_cols:
    full_df_plus4tuple[col+'_map'] = full_df_plus4tuple[col].map(dict_4heros)
    train_X_plus4tuple[col+'_map'] = train_X_plus4tuple[col].map(dict_4heros)

In [658]:
full_df_plus4tuple.drop(tuple_cols,inplace=True,axis=1)
train_X_plus4tuple.drop(tuple_cols,inplace=True,axis=1)

In [659]:
full_df = full_df_plus4tuple
train_X = train_X_plus4tuple

In [660]:
full_df.shape

(39675, 336)

In [661]:
train_X.shape

(39675, 335)

In [680]:
set(train_X.columns.values).difference(set(test_X_plus_4tuple.columns.values))

set()

Map `dict_4heros` to `test_X`

In [408]:
#set(full_df.columns.values).difference(set(test_X_plus_4tuple.columns.values))

In [409]:
#set(test_X_plus_4tuple.columns.values).difference(set(full_df.columns.values))

In [671]:
r_hero_cols =['r{}_hero_id'.format(i) for i in range(1,6)]
d_hero_cols =['d{}_hero_id'.format(i) for i in range(1,6)]
test_X_r_heros = test_X[r_hero_cols]
test_X_d_heros = test_X[d_hero_cols]
test_X_r_heros.values.sort()
test_X_d_heros.values.sort()
make_4tuple(test_X_r_heros,'r');
make_4tuple(test_X_d_heros,'d');
for col in tuple_cols_r:
    test_X_r_heros[col+'_map'] = test_X_r_heros[col].map(dict_4heros)
for col in tuple_cols_d:
    test_X_d_heros[col+'_map'] = test_X_d_heros[col].map(dict_4heros)

In [672]:
test_X_d_heros.fillna(0,inplace=True)
test_X_r_heros.fillna(0,inplace=True)

In [673]:
test_X_d_heros.columns

Index(['d1_hero_id', 'd2_hero_id', 'd3_hero_id', 'd4_hero_id', 'd5_hero_id',
       'd_hero_4tuple_1', 'd_hero_4tuple_2', 'd_hero_4tuple_3',
       'd_hero_4tuple_4', 'd_hero_4tuple_5', 'd_hero_4tuple_1_map',
       'd_hero_4tuple_2_map', 'd_hero_4tuple_3_map', 'd_hero_4tuple_4_map',
       'd_hero_4tuple_5_map'],
      dtype='object')

In [674]:
hero_id_list_d = ['d{}_hero_id'.format(i) for i in range(1,6)] 
hero_id_list_r =['r{}_hero_id'.format(i) for i in range(1,6)] 
test_X_d_heros.drop(hero_id_list_d,inplace=True,axis=1)
test_X_r_heros.drop(hero_id_list_r,inplace=True,axis=1)

In [675]:
tuple_cols_d_map = ['d_hero_4tuple_{}_map'.format(i) for i in range(1,6)] 
tuple_cols_r_map =['r_hero_4tuple_{}_map'.format(i) for i in range(1,6)] 

In [676]:
test_X_plus_4tuple = pd.concat([test_X,test_X_d_heros[tuple_cols_d_map]],axis=1)

In [677]:
test_X_plus_4tuple = pd.concat([test_X_plus_4tuple,test_X_r_heros[tuple_cols_r_map]],axis=1)

In [678]:
full_df.shape

(39675, 336)

In [681]:
test_X_plus_4tuple.shape

(10000, 335)

In [682]:
test_X = test_X_plus_4tuple

In [684]:
full_df.drop(hero_id_list,inplace=True,axis=1)
train_X.drop(hero_id_list,inplace=True,axis=1)
test_X.drop(hero_id_list,inplace=True,axis=1)

In [738]:
#same columns
set(full_df.columns.values).difference(set(train_X.columns.values))
set(train_X.columns.values).difference(set(full_df.columns.values))
set(test_X.columns.values).difference(set(train_X.columns.values))
set(train_X.columns.values).difference(set(test_X.columns.values))
set(full_df.columns.values).difference(set(test_X.columns.values))
set(test_X.columns.values).difference(set(full_df.columns.values))

set()

### 2 at a time

In [935]:
r_hero_ids_by2 = list(itertools.combinations(r_hero_ids, 2))
d_hero_ids_by2 = list(itertools.combinations(d_hero_ids, 2))
hero_ids_by2 = list(itertools.combinations(hero_ids_num,2))

In [947]:
new_hero_cols = []
for i in range(1,6):
    for j in range(i+1,6):
        new_hero_cols.append('r'+str(i)+'_'+str(j))
for i in range(1,6):
    for j in range(i+1,6):
        new_hero_cols.append('d'+str(i)+'_'+str(j))

In [967]:
hero_by2_dict_win = {k:0 for k in hero_ids_by2}
for i in range(r_heros_df_rad_win.shape[0]):
    for j in range(r_heros_df_rad_win.shape[1]):
        for k in range(j+1,r_heros_df_rad_win.shape[1]):
            Key = (r_heros_df_rad_win.iloc[i,j],r_heros_df_rad_win.iloc[i,k])
            hero_by2_dict_win[Key] +=1
for i in range(d_heros_df_rad_win.shape[0]):
    for j in range(d_heros_df_rad_win.shape[1]):
        for k in range(j+1,d_heros_df_rad_win.shape[1]):
            Key = (d_heros_df_rad_win.iloc[i,j],d_heros_df_rad_win.iloc[i,k])
            hero_by2_dict_win[Key] +=1

In [969]:
hero_by2_dict_lose = {k:0 for k in hero_ids_by2}
for i in range(r_heros_df_rad_lose.shape[0]):
    for j in range(r_heros_df_rad_lose.shape[1]):
        for k in range(j+1,r_heros_df_rad_lose.shape[1]):
            Key = (r_heros_df_rad_lose.iloc[i,j],r_heros_df_rad_lose.iloc[i,k])
            hero_by2_dict_lose[Key] +=1
for i in range(d_heros_df_rad_lose.shape[0]):
    for j in range(d_heros_df_rad_lose.shape[1]):
        for k in range(j+1,d_heros_df_rad_lose.shape[1]):
            Key = (d_heros_df_rad_lose.iloc[i,j],d_heros_df_rad_lose.iloc[i,k])
            hero_by2_dict_lose[Key] +=1

In [988]:
d1 = Counter(hero_by2_dict_win)
d2 = Counter(hero_by2_dict_lose)
d1.subtract(d2)
hero_by2_dict_net = d1

Now data frames have each row of hero ids sorted least to greatest

In [958]:
new_hero_cols

['r1_2',
 'r1_3',
 'r1_4',
 'r1_5',
 'r2_3',
 'r2_4',
 'r2_5',
 'r3_4',
 'r3_5',
 'r4_5',
 'd1_2',
 'd1_3',
 'd1_4',
 'd1_5',
 'd2_3',
 'd2_4',
 'd2_5',
 'd3_4',
 'd3_5',
 'd4_5']

In [ ]:
def find_2_heros(game):
    

Also add feature of normalized hero id (divide by total games for each hero)

Now have mapped hero id's to how often they participate in winning games, so will now be a numeric feature and can also use sum of this as a new feature.

## Using json files for new features: 

In [617]:
with open(os.path.join(PATH_TO_DATA, 'train_matches.jsonl')) as fin:
    # read the 18-th line
    for i in range(18):
        line = fin.readline()
    
    # read JSON into a Python object 
    match = json.loads(line)

In [619]:
match.keys()

dict_keys(['game_time', 'match_id_hash', 'teamfights', 'objectives', 'chat', 'game_mode', 'lobby_type', 'players', 'targets'])

### Working with ability upgrades

For a single match, test new features:
`max_ability_dt`
`min_ability_dt`
`mean_ability_dt`
`max_ability_da`
`min_ability_da`
`mean_ability_da`

In [661]:
player = match['players'][2]
n = len(player['ability_upgrades'])
first_ability_time = player['ability_upgrades'][0]['time']
last_ability_time = player['ability_upgrades'][n-1]['time']
ability_dt = [first_ability_time]
#time differences between level changes
for i in range(n-1):
    diff = player['ability_upgrades'][i+1]['time'] - player['ability_upgrades'][i]['time']
    ability_dt.append(diff)
min_ability_dt = min(ability_dt)
max_ability_dt = max(ability_dt)
mean_ability_dt = sum(ability_dt)/len(ability_dt)
print(min_ability_dt,mean_ability_dt,max_ability_dt,last_ability_time)
#ability differences between level changes
ability_d_ability = [player['ability_upgrades'][0]['ability']]
last_ability = player['ability_upgrades'][n-1]['ability']
for i in range(n-1):
    diff = player['ability_upgrades'][i+1]['ability'] - player['ability_upgrades'][i]['ability']
    ability_d_ability.append(diff)
min_ability_d_ability = min(ability_d_ability)
max_ability_d_ability = max(ability_d_ability)
mean_ability_d_ability = sum(ability_d_ability)/len(ability_d_ability)
print(min_ability_d_ability,mean_ability_d_ability,max_ability_d_ability,last_ability)

4 120.5625 227 1929
-1352 314.5625 5032 5033


In [663]:
#player['ability_upgrades']

In [717]:
def read_matches(matches_file):
    
    MATCHES_COUNT = {
        'test_matches.jsonl': 10000,
        'train_matches.jsonl': 39675,
    }
    _, filename = os.path.split(matches_file)
    total_matches = MATCHES_COUNT.get(filename)
    
    with open(matches_file) as fin:
        for line in tqdm_notebook(fin, total=total_matches):
            yield json.loads(line)

In [718]:
matches_file = '/Users/m/Insight/MLcourse/Dota/train_matches.jsonl'

In [791]:
def extract_features_csv(match):
    row = [
        ('match_id_hash', match['match_id_hash']),
    ]
    
    for slot, player in enumerate(match['players']):
        if slot < 5:
            player_name = 'r%d' % (slot + 1)
        else:
            player_name = 'd%d' % (slot - 4)
            
        n = len(player['ability_upgrades'])
        if n > 0:    
            first_ability_time = player['ability_upgrades'][0]['time']
            row.append((f'{player_name}_first_ability_t', first_ability_time))
            last_ability_time = player['ability_upgrades'][n-1]['time']
            row.append((f'{player_name}_last_ability_t', last_ability_time))
            ability_dt = [first_ability_time]
            #time differences between level changes
            for i in range(n-1):
                diff = player['ability_upgrades'][i+1]['time'] - player['ability_upgrades'][i]['time']
                ability_dt.append(diff)
            min_ability_dt = min(ability_dt)
            row.append((f'{player_name}_min_ability_dt', min_ability_dt))
            max_ability_dt = max(ability_dt)
            row.append((f'{player_name}_max_ability_dt', max_ability_dt))
            mean_ability_dt = sum(ability_dt)/len(ability_dt)
            row.append((f'{player_name}_mean_ability_dt', mean_ability_dt))

            #ability differences between level changes
            first_ability = player['ability_upgrades'][0]['ability']
            row.append((f'{player_name}_first_ability', first_ability))
            last_ability = player['ability_upgrades'][n-1]['ability']
            row.append((f'{player_name}_last_ability', last_ability))
            dability = []
            for i in range(n-1):
                diff = player['ability_upgrades'][i+1]['ability'] - player['ability_upgrades'][i]['ability']
                dability.append(diff)
            try:
                min_ability_da = min(dability)
            except:
                min_ability_da = 0
            row.append((f'{player_name}_min_ability_da', min_ability_da))
            try:
                max_ability_da = max(dability)
            except:
                max_ability_da = 0
            row.append((f'{player_name}_max_ability_da', max_ability_da))
            try:
                mean_ability_da = sum(dability)/len(dability)
            except:
                mean_ability_da = 0
            row.append((f'{player_name}_mean_ability_da', mean_ability_da))
        else:
            row.append((f'{player_name}_first_ability_t', 0))
            row.append((f'{player_name}_last_ability_t', 0))
            row.append((f'{player_name}_min_ability_dt', 0))
            row.append((f'{player_name}_max_ability_dt', 0))
            row.append((f'{player_name}_mean_ability_dt', 0))

            row.append((f'{player_name}_first_ability', 0))
            row.append((f'{player_name}_last_ability', 0))
            row.append((f'{player_name}_min_ability_da', 0))
            row.append((f'{player_name}_max_ability_da', 0))
            row.append((f'{player_name}_mean_ability_da', 0))
    
    return collections.OrderedDict(row)

In [792]:
json_feats = []
for match in read_matches(matches_file):
    match_id_hash = match['match_id_hash']
    features = extract_features_csv(match)
    json_feats.append(features)
df_json_feats = pd.DataFrame(json_feats).set_index('match_id_hash')

In [794]:
#export_csv = df_json_feats.to_csv(r'/Users/m/Insight/MLcourse/Dota/train_features_ability.csv', index = 'match_id_hash', header=True)

In [796]:
matches_file_test = '/Users/m/Insight/MLcourse/Dota/test_matches.jsonl'

In [797]:
json_feats_test = []
for match in read_matches(matches_file_test):
    match_id_hash = match['match_id_hash']
    features = extract_features_csv(match)
    json_feats_test.append(features)
df_json_feats_test = pd.DataFrame(json_feats_test).set_index('match_id_hash')

In [798]:
#export_csv_test = df_json_feats_test.to_csv(r'/Users/m/Insight/MLcourse/Dota/test_features_ability.csv', index = 'match_id_hash', header=True)

In [767]:
#test on a few features
matches = []

with open(os.path.join('/Users/m/Insight/MLcourse/Dota/train_matches.jsonl')) as fin:
    for j in range(5):
        for i in range(j):
            line = fin.readline()

        # read JSON into a Python object 
        matches.append(json.loads(line))

len(matches)

5

In [786]:
json_feats = []
for match in matches:
    match_id_hash = match['match_id_hash']
    features = extract_features_csv(match)
    json_feats.append(features)
DF = pd.DataFrame(json_feats).set_index('match_id_hash')
DF.head()

,r1_first_ability_t,r1_last_ability_t,r1_min_ability_dt,r1_max_ability_dt,r1_mean_ability_dt,r1_first_ability,r1_last_ability,r1_min_ability_da,r1_max_ability_da,r1_mean_ability_da,...,d5_first_ability_t,d5_last_ability_t,d5_min_ability_dt,d5_max_ability_dt,d5_mean_ability_dt,d5_first_ability,d5_last_ability,d5_min_ability_da,d5_max_ability_da,d5_mean_ability_da
match_id_hash,,,,,,,,,,,,,,,,,,,,,
7469e9440ea3d19b211a484647c7126e,230,984,76,262,164.0,5353,5356,-2,2,0.6,...,228,1033,47,228,103.3,5098,6926,-2,1828,203.111111
a400b8f29dece5f4d266f49f1ae2e98a,0,0,0,0,0.0,0,0,0,0,0.0,...,0,0,0,0,0.0,0,0,0,0,0.000000
6db558535151ea18ca70a6892197db41,0,0,0,0,0.0,0,0,0,0,0.0,...,0,0,0,0,0.0,0,0,0,0,0.000000
19c39fe2af2b547e48708ca005c6ae74,0,0,0,0,0.0,0,0,0,0,0.0,...,0,0,0,0,0.0,0,0,0,0,0.000000
7469e9440ea3d19b211a484647c7126e,230,984,76,262,164.0,5353,5356,-2,2,0.6,...,228,1033,47,228,103.3,5098,6926,-2,1828,203.111111


## Find new features:

1. Use data from `match['teamfights']`. This could be hard because nothing specific here about  teams, would have to link teamfights to players.
2. Hero data:
Find best hero combinations for winning games

### Creating new features

In [685]:
test_X.shape

(10000, 325)

In [686]:
train_X.shape

(39675, 325)

In [687]:
full_df.shape

(39675, 326)

In [688]:
train_test_feat = pd.concat([full_df.drop('radiant_win',axis=1),test_X])
train_test_feat.shape

(49675, 325)

In [741]:
set(train_test_feat.columns.values).difference(set(full_df.columns.values))
set(train_X.columns.values).difference(set(train_test_feat.columns.values))

set()

In [742]:
def select_cols_n_distinct(data,min,max):
    '''select columns in data frame with number of 
    distinct values between min and max, inclusive'''
    cols = []
    for col in data.columns.values:
        unique_vals = data[col].value_counts().shape[0]
        if unique_vals >= min and unique_vals <= max:
            #cols.append((col,unique_vals))
            cols.append(col)
    return cols    

In [743]:
select_cols_n_distinct(full_df.drop('radiant_win',axis=1),2,2);

In [744]:
radiant_win_T = full_df[full_df['radiant_win'] == 1]['lobby_type']
radiant_win_F = full_df[full_df['radiant_win'] == 0]['lobby_type']
#sns.distplot(radiant_win_T, bins=100, label='R won')
#sns.distplot(radiant_win_F, bins=100, label='R lost')
#plt.xlim(1900, 2020)
#plt.legend()

In [745]:
#sns.countplot(x="lobby_type", hue="radiant_win", data=full_df);

In [746]:
new_features = pd.DataFrame(index=train_test_feat.index)

In [747]:
new_feats = [i.replace("1","")  for i in full_df.columns.values if (i.startswith('r1')==1 
                                                                    or i.startswith('d1')==1)]


Create a feature that is sum of r1 + ... + r5 for all r features, and same for all d features. Note that not all of these make sense, so some should be removed.

In [748]:
for feature in new_feats:
    feat_names = [(feature[0] + '{}_'.format(i) +feature[2:]) for i in range(1,6)]
    new_features[feature] = train_test_feat.loc[:,feat_names].sum(axis=1)
   # new_features[feature + '_mean'] = train_test_feat.loc[:,feat_names].mean(axis=1)
   #new_features[feature + '_std'] = train_test_feat.loc[:,feat_names].std(axis=1)
#new_features.drop(['r_hero_id','d_hero_id'],inplace=True,axis=1)

In [749]:
len(new_features.columns)
#new_features.isnull().sum().sum()

62

### New features from TK

In [750]:
health = ['r{}_health'.format(x) for x in range(1,6)] +['d{}_health'.format(x) for x in range(1,6)]
max_health = ['r{}_health'.format(x) for x in range(1,6)] +['d{}_health'.format(x) for x in range(1,6)]
x = ['r{}_x'.format(x) for x in range(1,6)] +['d{}_x'.format(x) for x in range(1,6)]
y1 = ['r{}_y'.format(x) for x in range(1,6)] +['d{}_y'.format(x) for x in range(1,6)]
xp = ['r{}_xp'.format(x) for x in range(1,6)] +['d{}_xp'.format(x) for x in range(1,6)]

train_positions = train_X[health+max_health+x+y1+xp]
#train_positions.head(2)

In [751]:
def are_dead(x):
    ans = 0 
    for el in x:
        if el == 0:
            ans += 1
    return ans

In [752]:
d_health = ['d{}_health'.format(i) for i in range(1,6)]
new_features['d_dead'] = full_df[d_health].apply(are_dead, axis = 1)
r_health = ['r{}_health'.format(i) for i in range(1,6)]
new_features['r_dead'] = full_df[r_health].apply(are_dead, axis = 1)


In [753]:
percentage_health = 0
for j in range(5):
    max_health = '{}_max_health'.format(d_health[j].split('_')[0])
    health = 'd{}_health'.format(j+1)
    ph = full_df[health]/full_df[max_health]
    percentage_health += 1/5*ph

new_features['d_health_avg'] = percentage_health

percentage_health = 0
for j in range(5):
    max_health = '{}_max_health'.format(r_health[j].split('_')[0])
    health = 'r{}_health'.format(j+1)
    ph = full_df[health]/full_df[max_health]
    percentage_health += 1/5*ph

new_features['r_health_avg'] = percentage_health

In [754]:


def in_opponent_base(x,y,opponent = 'Radiant'):

    '''
returns indicator function of whether the player is in the base of the opponent
opponent = {'Radiant', 'Dire'} - opponent team
    
    '''
#     x = coordinates[0]
#     y = coordinates[1]
    radiant_base_x = 96
    radiant_base_y = 100

    dire_base_x = 156
    dire_base_y = 156

    if opponent == 'Radiant':
        if x <= radiant_base_x and y <= radiant_base_y:
            return 1
        else:
            return 0
    elif opponent == 'Dire':
        if x >= dire_base_x and y >= dire_base_y:
            return 1
        else:
            return 0
    else:
        return NaN



In [755]:


r_x = ['r{}_x'.format(j) for j in range(1,6)]
r_y = ['r{}_y'.format(j) for j in range(1,6)]
d_x = ['d{}_x'.format(j) for j in range(1,6)]
d_y = ['d{}_y'.format(j) for j in range(1,6)]

r_in_d_base = pd.Series(0,index = full_df.index)
d_in_r_base = pd.Series(0,index = full_df.index)

for j in range(5):
    rx = r_x[j]
    ry = r_y[j]
    dx = d_x[j]
    dy = d_y[j]
    r_in_d_base += full_df.loc[:,[rx,ry]].apply(lambda x: in_opponent_base(x = x[rx], y = x[ry], opponent = 'Radiant'), axis = 1)
    d_in_r_base += full_df.loc[:,[dx,dy]].apply(lambda x: in_opponent_base(x = x[dx], y = x[dy], opponent = 'Dire'), axis = 1)



In [756]:
new_features['r_in_d_base'] = r_in_d_base
new_features['d_in_r_base'] = d_in_r_base

In [757]:
d_levels = ['d{}_level'.format(j) for j in range(1,6)] 
d_avg_level = full_df.loc[:,d_levels].mean(axis = 1)
d_min_level = full_df.loc[:,d_levels].min(axis = 1)
d_max_level = full_df.loc[:,d_levels].max(axis = 1)
new_features['d_avg_level'] = d_avg_level
new_features['d_min_level'] = d_min_level
new_features['d_max_level'] = d_max_level


r_levels = ['r{}_level'.format(j) for j in range(1,6)] 
r_avg_level = full_df.loc[:,r_levels].mean(axis = 1)
r_min_level = full_df.loc[:,r_levels].min(axis = 1)
r_max_level = full_df.loc[:,r_levels].max(axis = 1)
new_features['r_avg_level'] = r_avg_level
new_features['r_min_level'] = r_min_level
new_features['r_max_level'] = r_max_level

In [758]:
r_teamfight = ['r{}_teamfight_participation'.format(j) for j in range(1,6)] 
for col in r_teamfight:
    full_df[full_df[col] >1][col] = 1

d_teamfight = ['d{}_teamfight_participation'.format(j) for j in range(1,6)] 
for col in d_teamfight:
    full_df[full_df[col] >1][col] = 1

In [759]:
#new_features.drop('target',axis=1).hist(figsize = (25,80),layout = (50,5), bins=20)
#new_features.hist(figsize = (25,80),layout = (60,5), bins=20)
#plt.tight_layout()
#plt.show()

### Create boolean features:

`*_ability_level`
`*_teamfight_participation`
`*_kills`
`*_assists`
`*_deaths`
`*_sen_placed`

In [760]:
test = pd.DataFrame()
def create_boolean_features(cols):
    for col in cols:
        new_features['boolean_' + col] = np.where(new_features[col]==0, 1, 0)

In [761]:
boolean_cols = ['r_ability_level','d_ability_level','r_teamfight_participation',
                'd_teamfight_participation','r_kills','d_kills',
                'r_assists','d_assists','r_deaths','d_deaths','r_sen_placed',
               'd_sen_placed']
create_boolean_features(boolean_cols)

### Take log of skewed features

In [762]:
import math
col_for_log =  ['r_count_ability_use','r_max_hero_hit',
                'r_damage_received','r_damage_dealt',
                'r_max_mana','r_max_health',
                'r_lh','r_health',
     'r_gold', 'r_xp', 'r_stuns',
    'd_count_ability_use', 'd_max_hero_hit',
                'd_max_mana','d_max_health',
                'd_damage_received','d_damage_dealt',
               'd_health', 'd_stuns',
     'd_gold', 'd_lh', 'd_xp']

for col in col_for_log:
    new_log_index = 'log_{}'.format(col)
#     print(new_features[col].min())
    new_features[col] = new_features[col].apply(lambda x: math.log(x+5))
    
#train_test_feat['chat_len'] = train_test_feat['chat_len'].apply(lambda x: math.log(x+5))

Conclusions about categorical variables:
* firstblood_claimed : categorical, binary
* game_mode: categorical, non-ordinal --> needs to be converted to dummy variables
* lobby_type: categorical, binary (value 7 chould be changes with 1)
* hero_id: categorical, non-ordinal (too many levels)
* level: categorical, ordinal (25 levels)
* x,y are coordinates of players

## Combine old and new features

In [710]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [855]:
train_test_feat.shape

(49675, 325)

In [764]:
new_features.shape

(49675, 86)

In [765]:
all_feats = train_test_feat.merge(new_features, how='outer', left_index=True, right_index = True)

In [766]:
all_feats.drop(all_feats.columns[all_feats.isnull().any()],inplace=True,axis=1)

In [767]:
all_feats.isnull().sum().sum()

0

In [768]:
all_feats.shape

(49675, 399)

### Split `all_features` into train and test data

In [769]:
ind = train_X.shape[0]
all_feats_train = all_feats.iloc[:ind,]
all_feats_test = all_feats.iloc[ind:,:]
print(all_feats_train.shape)
print(all_feats_test.shape)

(39675, 399)
(10000, 399)


In [777]:
all_feats_full = pd.concat([all_feats_train,y],axis=1)
all_feats_full.head(1)

,chat_len,d1_ability_level,d1_assists,d1_camps_stacked,d1_count_ability_use,d1_creeps_stacked,d1_damage_dealt,d1_damage_received,d1_deaths,d1_denies,...,boolean_d_teamfight_participation,boolean_r_kills,boolean_d_kills,boolean_r_assists,boolean_d_assists,boolean_r_deaths,boolean_d_deaths,boolean_r_sen_placed,boolean_d_sen_placed,radiant_win
0000c270e25494c03d5c81036463fc45,3,11,1,0,111,0,94692,10497,2,21,...,0,0,0,0,0,0,0,0,0,True


In [781]:
X_train, X_test, y_train, y_test = train_test_split(all_feats_full.drop('radiant_win',axis=1),
                                                    all_feats_full['radiant_win'], test_size=.3, 
                                                    stratify=all_feats_full['radiant_win'])

In [783]:
X_train.shape

(27772, 399)

In [784]:
y_train.shape

(27772,)

In [785]:
pd.DataFrame(y_train).isnull().sum()

radiant_win    0
dtype: int64

In [295]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer, LabelEncoder
from sklearn_pandas import DataFrameMapper

In [807]:
#hero_ids = ['r{}_hero_id'.format(i) for i in range(1,6)] +['d{}_hero_id'.format(i) for i in range(1,6)]
cat_feats = select_cols_n_distinct(all_feats_full,2,3)# + hero_ids
cat_feats.remove('radiant_win')
cat_feats.append('game_mode')

#cat_feats

In [808]:
#ord_feats = select_cols_n_distinct(all_feats,4,30)
#ord_feats.remove('game_mode')

In [809]:
num_feats = [feature for feature in all_feats_full.columns.values if feature not in cat_feats]
num_feats.remove('radiant_win')
#num_feats = [feature for feature in all_feats.columns.values if feature not in ord_feats]

In [166]:
from sklearn.ensemble import AdaBoostClassifier

In [167]:
from lightgbm import LGBMClassifier
from sklearn.base import BaseEstimator

In [810]:
class ClassifierCustom(BaseEstimator):
    def __init__(self, C=1, n_estimators=200):
        self.n_estimators = n_estimators
        self.C = C
        
        self.models = []
        #self.boot = []
        #self.feat_ids_by_tree = []
    def fit(self, X, y):
            #log reg
            lr = LogisticRegression(C = self.C)
            self.models.append(lr.fit(X,y))
            
            #Adaboost
            ab = AdaBoostClassifier(n_estimators = 250, learning_rate = 1)
            self.models.append(ab.fit(X,y))
            
            #lgbm
            lgbm = LGBMClassifier(n_jobs=-1)
            self.models.append(lgbm.fit(X,y))
            return self
    
    def predict_proba(self, X):
        self.probs = [self.models[0].predict_proba(X)] #weight logistic regression double
        for i in range(len(self.models)):
            self.probs.append(self.models[i].predict_proba(X))
        self.probs = np.array(self.probs)
        self.averages = np.mean(self.probs, axis = 0)
        return self.averages
    
    def predict(self,X):
        self.predictions =  self.predict_proba(X) >=0.5 
        return self.predictions


In [811]:
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier

class Columns(BaseEstimator, TransformerMixin):
    def __init__(self, names=None):
        self.names = names

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X):
        return X[self.names]
    
    

pipe = Pipeline([
    ("features", FeatureUnion([
        ('numeric', make_pipeline(Columns(names=num_feats),StandardScaler())),
        ('categorical', make_pipeline(Columns(names=cat_feats),OneHotEncoder(sparse=False)))
    ])),
  #  ('PCA',PCA(n_components = 300,whiten=True)),
    ('logreg', LogisticRegression(C=1))])
    #('custom', ClassifierCustom(C = 0.95, n_estimators = 250))])
    #('adaboost', AdaBoostClassifier(n_estimators = 150))])
       #('lgbm', LGBMClassifier(n_jobs=-1,boost = 'gbdt',
       #   feature_fraction= 0.05,
       #   learning_rate= 0.005,
       #     num_iterations = 500,
       #   max_depth= -1,  
       #   min_data_in_leaf = 150,
       #   num_leaves = 64,
       #   num_threads = -1,
       #   verbosity= 1,
       #   objective = 'binary'))])

In [819]:
set(train_test_feat.columns.values).difference(set(all_feats_train.columns.values))

set()

In [820]:
pipe.fit(all_feats_train,y)
#pipe.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('numeric', Pipeline(memory=None,
     steps=[('columns', Columns(names=['chat_len', 'd1_ability_level', 'd1_assists', 'd1_camps_stacked', 'd1_count_ability_use', 'd1_creeps_stacked', 'd1_damage_dealt', 'd1_damage_received', 'd1_dea...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [813]:
predicted = pipe.predict(X_test)

In [814]:
print(accuracy_score(y_test, predicted))

0.9945391918003864


In [815]:
print(roc_auc_score(y_test,predicted))

0.994529800376085


Note: So far, logreg with C = 0.95 does the best. The Custom ensemble is also close. PCA decreases both metrics and position on leaderboard; RF also decreased leaderboard position, over logreg (best so far).\
`cross_val_predict` gives ROC AUC 0.8132 for logistic regression. ROC AUC 0.8049 for lightGBM (untuned)

### Custom classifier: logreg, Adaboost, lgbm

## Grid Search

In [54]:
from sklearn.model_selection import GridSearchCV

In [80]:
param_grid_lgbm = {
    'lgbm__max_depth': [18],
    'lgbm__learning_rate': [.01],
    'lgbm__num_leaves': [4,8,16], #large will increase accuracy but may overfit
    'lgbm__learning_rate': [0.005],
    'lgbm__n_estimators': [40],
    'lgbm__num_leaves': [6,8,12,16],
    'lgbm__boosting_type' : ['gbdt'],
    'lgbm__objective' : ['binary'],
    'lgbm__random_state' : [501], # Updated from 'seed'
    'lgbm__colsample_bytree' : [0.65, 0.66],
    'lgbm__subsample' : [0.7,0.75],
    'lgbm__reg_alpha' : [1,1.2],
    'lgbm__reg_lambda' : [1,1.2,1.4],
    }

In [55]:
param_grid_logreg = {
    'logreg__C': [.9,0.95,1],
    'PCA__n_components': [300]
}

In [77]:
param_grid_ab = {
    'adaboost__n_estimators': [125,150,175],
}

In [62]:
grid_clf = GridSearchCV(pipe, param_grid_logreg, cv=5, iid=False)
grid_clf.fit(X_train, y_train);

In [63]:
grid_clf.best_params_

{'PCA__n_components': 300, 'logreg__C': 1}

In [66]:
print("grid search best: %f" % grid_clf.best_estimator_.score(X_test, y_test))


NameError: name 'y_test' is not defined

Logistic regression is doing better than other models. In grid search with best C = 0.9.

PCA best around 300 (+-15).

AdaBoost is doing more poorly; best `n_estimators` is 150 among `[50,100,150]`

### Preparing submission

In [821]:
#X_test_vals = all_feats_test.values
y_test_pred = pipe.predict_proba(all_feats_test)[:, 1]

df_submission = pd.DataFrame({'radiant_win_prob': y_test_pred}, 
                                 index=all_feats_test.index)


In [829]:
df_submission.index.name='match_id_hash'

In [830]:
df_submission.head(3)

,radiant_win_prob
match_id_hash,
30cc2d778dca82f2edb568ce9b585caa,0.482591
70e5ba30f367cea48793b9003fab9d38,0.488290
4d9ef74d3a2025d79e9423105fd73d41,0.562868


In [823]:
import datetime
submission_filename = 'submission_{}.csv'.format(
    datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
df_submission.to_csv(submission_filename)
print('Submission saved to {}'.format(submission_filename))

Submission saved to submission_2019-11-15_14-44-14.csv


Not using this:

In [86]:
from sklearn.model_selection import cross_val_predict

In [60]:
cv_pred = cross_val_predict(pipe, 
                             X_train, 
                             y_train, 
                             cv=5,
                             method="predict_proba")

In [61]:
print("Cross validation AUC {:.4f}".format(roc_auc_score(y_train, cv_pred[:,1])))

Cross validation AUC 0.8049


In [56]:
from sklearn.compose import ColumnTransformer, make_column_transformer

preprocess = make_column_transformer(
    (num_feats, StandardScaler()),
    (cat_feats, OneHotEncoder()))



/anaconda3/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:751: DeprecationWarning: `make_column_transformer` now expects (transformer, columns) as input tuples instead of (columns, transformer). This has been introduced in v0.20.1. `make_column_transformer` will stop accepting the deprecated (columns, transformer) order in v0.22.
  warnings.warn(message, DeprecationWarning)


In [58]:
pipe2 = make_pipeline(
    preprocess,
   # StandardScaler(),
    LGBMClassifier(n_jobs=-1)
)

## EDA + FE

In [577]:
full_df.columns

Index(['game_time', 'game_mode', 'lobby_type', 'objectives_len', 'chat_len',
       'r1_kills', 'r1_deaths', 'r1_assists', 'r1_denies', 'r1_gold',
       ...
       'r1_hero_idnorm', 'r2_hero_idnorm', 'r3_hero_idnorm', 'r4_hero_idnorm',
       'r5_hero_idnorm', 'd1_hero_idnorm', 'd2_hero_idnorm', 'd3_hero_idnorm',
       'd4_hero_idnorm', 'd5_hero_idnorm'],
      dtype='object', length=316)

In [578]:
full_df['game_mode'].value_counts()

22    31762
4      3564
23     2546
3      1200
2       408
5       188
12        5
16        2
Name: game_mode, dtype: int64

In [579]:
game_mode_rad_win = full_df[full_df['radiant_win'] == True]['game_mode']
game_mode_rad_lose = full_df[full_df['radiant_win'] == False]['game_mode']

In [581]:
game_mode_rad_win.value_counts(normalize=True)

22    0.803659
4     0.087871
23    0.064007
3     0.029434
2     0.010708
5     0.004177
12    0.000096
16    0.000048
Name: game_mode, dtype: float64

In [582]:
game_mode_rad_lose.value_counts(normalize=True)

22    0.797125
4     0.091994
23    0.064354
3     0.031142
2     0.009815
5     0.005358
12    0.000159
16    0.000053
Name: game_mode, dtype: float64

In [583]:
lobby_type_rad_win = full_df[full_df['radiant_win'] == True]['lobby_type']
lobby_type_rad_lose = full_df[full_df['radiant_win'] == False]['lobby_type']
print(lobby_type_rad_win.value_counts(normalize=True))
print(lobby_type_rad_lose.value_counts(normalize=True))

7    0.683569
0    0.316431
Name: lobby_type, dtype: float64
7    0.679771
0    0.320229
Name: lobby_type, dtype: float64


In [586]:
objectives_len_rad_win = full_df[full_df['radiant_win'] == True]['objectives_len']
objectives_len_rad_lose = full_df[full_df['radiant_win'] == False]['objectives_len']
#print(objectives_len_rad_win.value_counts(normalize=True))
#print(objectives_len_rad_lose.value_counts(normalize=True))

In [587]:
full_rad_win = full_df[full_df['radiant_win']==True]
full_rad_lose = full_df[full_df['radiant_win']==False]